# EDA dữ liệu Google Maps (Đà Nẵng)

Notebook này thực hiện:
- Đọc JSON gốc từ SerpApi và/hoặc CSV đã làm phẳng
- Thống kê số lượng kết quả theo từ khoá
- Phát hiện trùng lặp theo toạ độ (lat, lon) và theo `place_id`
- Loại bỏ trùng và lưu CSV cuối cùng


In [1]:
import os
import json
import pandas as pd
import numpy as np
from collections import Counter

INPUT_JSON = './data/danang_places_results.json'
RAW_CSV = './data/danang_places_results_raw.csv'
DEDUP_CSV = './data/danang_places_results_dedup.csv'
FINAL_CSV = './data/danang_places_results_final.csv'

print('Paths:')
print(' JSON :', os.path.abspath(INPUT_JSON))
print(' RAW  :', os.path.abspath(RAW_CSV))
print(' DEDUP:', os.path.abspath(DEDUP_CSV))
print(' FINAL:', os.path.abspath(FINAL_CSV))


Paths:
 JSON : /Users/soc_036/work_dir/study_dir/pbl_6/crawl_data/data/danang_places_results.json
 RAW  : /Users/soc_036/work_dir/study_dir/pbl_6/crawl_data/data/danang_places_results_raw.csv
 DEDUP: /Users/soc_036/work_dir/study_dir/pbl_6/crawl_data/data/danang_places_results_dedup.csv
 FINAL: /Users/soc_036/work_dir/study_dir/pbl_6/crawl_data/data/danang_places_results_final.csv


In [2]:
# Đọc JSON và làm phẳng nhanh (nếu chưa có CSV thô)

def normalize_local_result(keyword: str, item: dict) -> dict:
    gps = item.get('gps_coordinates') or {}
    operating_hours = item.get('operating_hours') or {}
    service_options = item.get('service_options') or {}
    types = ', '.join(item.get('types') or []) if isinstance(item.get('types'), list) else item.get('types')
    type_ids = ', '.join(item.get('type_ids') or []) if isinstance(item.get('type_ids'), list) else item.get('type_ids')
    return {
        'keyword': keyword,
        'position': item.get('position'),
        'title': item.get('title'),
        'place_id': item.get('place_id'),
        'data_id': item.get('data_id'),
        'data_cid': item.get('data_cid'),
        'provider_id': item.get('provider_id'),
        'rating': item.get('rating'),
        'reviews': item.get('reviews'),
        'price': item.get('price'),
        'type': item.get('type'),
        'types': types,
        'type_id': item.get('type_id'),
        'type_ids': type_ids,
        'address': item.get('address'),
        'open_state': item.get('open_state') or item.get('hours'),
        'phone': item.get('phone'),
        'website': item.get('website'),
        'latitude': gps.get('latitude'),
        'longitude': gps.get('longitude'),
        'user_review': item.get('user_review'),
        'thumbnail': item.get('thumbnail'),
        'serpapi_thumbnail': item.get('serpapi_thumbnail'),
        'reviews_link': item.get('reviews_link'),
        'photos_link': item.get('photos_link'),
        'place_id_search': item.get('place_id_search'),
        'operating_hours_json': json.dumps(operating_hours, ensure_ascii=False) if operating_hours else None,
        'service_options_json': json.dumps(service_options, ensure_ascii=False) if service_options else None,
    }

if not os.path.exists(RAW_CSV):
    with open(INPUT_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)
    rows = []
    for kw, payload in data.items():
        for item in payload.get('local_results') or []:
            rows.append(normalize_local_result(kw, item))
    df_raw = pd.DataFrame(rows)
    print('Lưu RAW_CSV:', RAW_CSV)
    df_raw.to_csv(RAW_CSV, index=False)
else:
    df_raw = pd.read_csv(RAW_CSV)

df_raw.head()


,keyword,position,title,place_id,data_id,data_cid,provider_id,rating,reviews,price,...,latitude,longitude,user_review,thumbnail,serpapi_thumbnail,reviews_link,photos_link,place_id_search,operating_hours_json,service_options_json
0,Quán ăn Đà Nẵng,1,Cơm nhà vui,ChIJi5XzTqsZQjERn5HbwcdvD5Y,0x314219ab4ef3958b:0x960f6fc7c1db919f,10812984134081417631,/g/11k02pk2dj,4.9,2153.0,100-200 N ₫,...,16.056618,108.222067,"""Vị đồ ăn các món đều khá ngon, được tặng kèm ...",https://lh3.googleusercontent.com/p/AF1QipMfwK...,https://serpapi.com/images/url/M1hg0nicuxmXUVJ...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""10:00–22:00"", ""thứ ba"": ""10:00–22...","{""Ăn_tại_chỗ"": true, ""Đồ_ăn_mang_đi"": true, ""g..."
1,Quán ăn Đà Nẵng,2,Hải sản Mộc quán Đà Nẵng,ChIJx6-KGnIXQjER7x-T0zeOlBw,0x314217721a8aafc7:0x1c948e37d3931fef,2059427300039139311,/g/11h64qj0wn,4.7,15505.0,$$,...,16.063996,108.241574,"""Quán đồ ăn cực ngon vị trí đẹp có nhiều view ...",https://lh3.googleusercontent.com/p/AF1QipPtOs...,https://serpapi.com/images/url/j_5ADHicuxmXUVJ...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""10:30–22:30"", ""thứ ba"": ""10:30–22...","{""Ăn_tại_chỗ"": true, ""Đồ_ăn_mang_đi"": true, ""g..."
2,Quán ăn Đà Nẵng,3,Cô Ba Phở bò,ChIJQYGTw4MZQjER9-WZcXZUfAM,0x31421983c3938141:0x37c54767199e5f7,251168546914690551,/g/11gnpx_f59,4.8,6552.0,100-200 N ₫,...,16.066971,108.224719,"""Mình thấy mang đồ ăn ra khá nhanh, không phải...",https://lh3.googleusercontent.com/gps-cs-s/AC9...,https://serpapi.com/images/url/cf1Oe3icBcFRboI...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""10:00–22:00"", ""thứ ba"": ""10:00–22...","{""Ăn_tại_chỗ"": true, ""Đồ_ăn_mang_đi"": true, ""g..."
3,Quán ăn Đà Nẵng,4,Bếp Cuốn Đà Nẵng,ChIJ-UgnY_QXQjERJTTHpNvcFNw,0x314217f4632748f9:0xdc14dcdba4c73425,15858543023798826021,/g/11hz90gsyw,4.9,11874.0,100-300 N ₫,...,16.057963,108.245150,"""Quán view đẹp, đồ ăn ngon, đồ uống cũng ngon ...",https://lh3.googleusercontent.com/p/AF1QipNFjF...,https://serpapi.com/images/url/dZKG3XicuxmXUVJ...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""10:30–22:00"", ""thứ ba"": ""10:30–22...","{""Ăn_tại_chỗ"": true, ""Đồ_ăn_mang_đi"": true, ""g..."
4,Quán ăn Đà Nẵng,5,Bánh Xèo Bà Dưỡng,ChIJrSiIJ8oZQjERn2-PvhzOCuU,0x314219ca278828ad:0xe50ace1cbe8f6f9f,16504230407304081311,/g/11csqxxgqn,4.3,8810.0,1-100.000 ₫,...,16.058877,108.216166,"""Quán nằm trong hẻm nhỏ dài khoảng 100m không ...",https://lh3.googleusercontent.com/gps-cs-s/AC9...,https://serpapi.com/images/url/49otvXicBcFtDoI...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""09:30–21:30"", ""thứ ba"": ""09:30–21...","{""Ăn_tại_chỗ"": true, ""mua_hàng_ngay_trên_xe"": ..."


In [3]:
# Thống kê theo từ khoá

stats = (
    df_raw.groupby('keyword')
          .agg(n_records=('title', 'count'),
               n_unique_titles=('title', 'nunique'),
               avg_rating=('rating', 'mean'),
               n_has_phone=('phone', lambda s: s.notna().sum()),
               n_has_website=('website', lambda s: s.notna().sum()))
          .reset_index()
          .sort_values('n_records', ascending=False)
)

stats


,keyword,n_records,n_unique_titles,avg_rating,n_has_phone,n_has_website
0,BBQ Đà Nẵng,20,20,4.780000,20,17
1,Bars in Da Nang,20,20,4.715000,20,16
2,Best food Da Nang,20,20,4.750000,20,14
3,Buffet Đà Nẵng,20,20,4.575000,20,18
4,Bánh tráng cuốn thịt heo Đà Nẵng,20,20,4.345000,18,7
5,Bánh xèo Đà Nẵng,20,20,4.440000,20,7
6,Bún bò Đà Nẵng,20,20,4.370000,18,5
7,Bún chả cá Đà Nẵng,20,20,4.445000,18,8
8,Cao lầu Đà Nẵng,20,20,4.738889,11,2
9,Coffee Da Nang,20,20,4.585000,17,15


In [4]:
# Phát hiện trùng theo toạ độ (làm tròn 6 chữ số) và theo place_id

coord_round = 6

df_work = df_raw.copy()

df_work['lat_r'] = df_work['latitude'].round(coord_round)

df_work['lon_r'] = df_work['longitude'].round(coord_round)

# dấu hiệu trùng theo toạ độ
coord_dupe_mask = df_work.duplicated(subset=['lat_r', 'lon_r'], keep=False)

coord_groups = (
    df_work[coord_dupe_mask]
    .sort_values(['lat_r', 'lon_r'])
    .groupby(['lat_r', 'lon_r'])
)

print('Số nhóm toạ độ trùng:', coord_groups.ngroups)

# trùng theo place_id
placeid_dupe_mask = df_work.duplicated(subset=['place_id'], keep=False)

print('Số bản ghi trùng place_id:', int(placeid_dupe_mask.sum()))

# Loại trùng ưu tiên theo place_id trước, sau đó theo toạ độ
# Giữ lại bản ghi có nhiều reviews hơn (nếu có), rồi theo keyword đầu tiên

def pick_first(group: pd.DataFrame) -> pd.Series:
    # sort by reviews desc, then by keyword asc, then position asc
    sort_cols = [
        ('reviews', False),
        ('keyword', True),
        ('position', True),
    ]
    sort_by = [c for c, _ in sort_cols if c in group.columns]
    ascending = [asc for _, asc in sort_cols if _ in group.columns]
    return group.sort_values(by=sort_by, ascending=ascending).iloc[0]

# 1) Gộp theo place_id
if 'place_id' in df_work.columns:
    dedup_by_place = df_work.groupby('place_id', dropna=False, as_index=False).apply(pick_first).reset_index(drop=True)
else:
    dedup_by_place = df_work.copy()

# 2) Sau đó loại theo toạ độ
if {'latitude', 'longitude'}.issubset(dedup_by_place.columns):
    dedup_by_place['lat_r'] = dedup_by_place['latitude'].round(coord_round)
    dedup_by_place['lon_r'] = dedup_by_place['longitude'].round(coord_round)
    dedup_final = dedup_by_place.drop_duplicates(subset=['lat_r', 'lon_r'], keep='first').drop(columns=['lat_r', 'lon_r'])
else:
    dedup_final = dedup_by_place.copy()

print('Số dòng raw       :', len(df_raw))
print('Sau khi dedup pid :', len(dedup_by_place))
print('Sau khi dedup geo :', len(dedup_final))

dedup_final.head()


Số nhóm toạ độ trùng: 147
Số bản ghi trùng place_id: 402
Số dòng raw       : 806
Sau khi dedup pid : 549
Sau khi dedup geo : 547


/var/folders/yt/cbj8dfj102bb51m88fb5hw840000gn/T/ipykernel_7388/385003955.py:43: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dedup_by_place = df_work.groupby('place_id', dropna=False, as_index=False).apply(pick_first).reset_index(drop=True)


,keyword,position,title,place_id,data_id,data_cid,provider_id,rating,reviews,price,...,latitude,longitude,user_review,thumbnail,serpapi_thumbnail,reviews_link,photos_link,place_id_search,operating_hours_json,service_options_json
0,Bún bò Đà Nẵng,13,Bún Bò Huế Thượng Thành,ChIJ-3Ib2XAZQjERfgXezNQECUw,0x31421970d91b72fb:0x4c0904d4ccde057e,5478915733689599358,/g/11gmcdlxkf,4.4,237.0,1-100.000 ₫,...,16.046685,108.209874,NaN,https://lh3.googleusercontent.com/gps-cs-s/AC9...,https://serpapi.com/images/url/Cs2HKHicBcHdDoI...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""05:30–11:00"", ""thứ ba"": ""05:30–11...","{""Ăn_tại_chỗ"": true, ""mua_hàng_ngay_trên_xe"": ..."
1,KFC Đà Nẵng,4,KFC 21 Lê Duẩn,ChIJ-3x8RtwZQjERqRsqM1yvhiU,0x314219dc467c7cfb:0x2586af5c332a1ba9,2704041436812811177,/g/11tjs7cdly,4.6,562.0,100-200 N ₫,...,16.071383,108.221555,NaN,https://lh3.googleusercontent.com/gps-cs-s/AC9...,https://serpapi.com/images/url/tLVa53icBcHhtkI...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""10:00–22:00"", ""thứ ba"": ""10:00–22...","{""Ăn_tại_chỗ"": true, ""Đồ_ăn_mang_đi"": true, ""g..."
2,Bánh tráng cuốn thịt heo Đà Nẵng,2,Ẩm Thực Hoàng Tín - Bánh Tráng Cuốn Thịt Heo,ChIJ-5uyO8sZQjERdEXQ33dLdVM,0x314219cb3bb29bfb:0x53754b77dfd04574,6013795855651325300,/g/11cm7554_w,4.0,651.0,100-200 N ₫,...,16.054285,108.239823,"""Bánh tráng cuốn thịt heo ngon, thịt và rau rấ...",https://lh3.googleusercontent.com/p/AF1QipNRo0...,https://serpapi.com/images/url/pN45T3icuxmXUVJ...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""08:00–22:00"", ""thứ ba"": ""08:00–22...","{""Ăn_tại_chỗ"": true, ""mua_hàng_ngay_trên_xe"": ..."
3,Quán ăn chay Đà Nẵng,5,Bếp Chay Không | Khong Vegan Kitchen | 자작 국제적이...,ChIJ-8AB3oAXQjERptcQoIvuNCE,0x31421780de01c0fb:0x2134ee8ba010d7a6,2392799585493899174,/g/11j5672bmh,4.8,752.0,1-100.000 ₫,...,16.051968,108.243919,"""Món taco chay ở đây ngon cực, nhân viên dễ th...",https://lh3.googleusercontent.com/gps-cs-s/AC9...,https://serpapi.com/images/url/634KuXicBcFRkoI...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""10:00–22:30"", ""thứ ba"": ""10:30–22...","{""Ăn_tại_chỗ"": true, ""nhận_hàng_ở_lề_đường"": t..."
4,Phở Đà Nẵng,10,Phở Cổ,ChIJ-8ZiS8IZQjERZxAsTrpk3RM,0x314219c24b62c6fb:0x13dd64ba4e2c1067,1431411007911891047,/g/11skbq203t,4.7,594.0,1-100.000 ₫,...,16.065226,108.221385,NaN,https://lh3.googleusercontent.com/gps-cs-s/AC9...,https://serpapi.com/images/url/ZEhRpHicBcFRboI...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?data_id=0x3142...,https://serpapi.com/search.json?engine=google_...,"{""thứ hai"": ""07:00–01:00"", ""thứ ba"": ""07:00–01...","{""Ăn_tại_chỗ"": true, ""mua_hàng_ngay_trên_xe"": ..."


In [ ]:
# Lưu kết quả

# CSV đã loại trùng theo logic notebook (place_id -> geo)
print('Lưu DEDUP_CSV:', DEDUP_CSV)
dedup_final.to_csv(DEDUP_CSV, index=False)

# CSV cuối cùng: có thể chính là DEDUP_CSV, hoặc có lọc thêm điều kiện (vd: có rating)
df_final = dedup_final.copy()
print('Lưu FINAL_CSV:', FINAL_CSV)
df_final.to_csv(FINAL_CSV, index=False)

len(df_final)


Lưu DEDUP_CSV: ./data/danang_places_results_dedup.csv
Lưu FINAL_CSV: ./data/danang_places_results_final.csv


547